<a href="https://colab.research.google.com/github/sousci/myColab_GREEN-DATA-Challenge-2025/blob/main/20250602.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A100使用

## 🔰 Step 0. Colab用セットアップ

In [1]:
!pip install lightgbm

## 📥 Step 1. データの読み込みと前処理

In [2]:
import pandas as pd
import numpy as np

# データ読み込み
PATH = '/content/drive/MyDrive/SIGNATE/SMBC_Group_GREENDATA_Challenge_2025/'
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')
feature_desc = pd.read_csv(PATH + 'feature_description.csv')

In [3]:
# trainの前処理（price_actualは除く）
train_sorted = train.sort_values("time").reset_index(drop=True)
numeric_cols = train_sorted.select_dtypes(include='number').columns.tolist()
numeric_cols.remove("price_actual")  # 目的変数は除外

# expanding標準化の適用
for col in numeric_cols:
    train_sorted[f"{col}_mean_to_t"] = train_sorted[col].expanding().mean()
    train_sorted[f"{col}_std_to_t"] = train_sorted[col].expanding().std(ddof=0)
    train_sorted[f"{col}_scaled"] = (
        (train_sorted[col] - train_sorted[f"{col}_mean_to_t"]) /
        train_sorted[f"{col}_std_to_t"]
    )

# 標準化済みの特徴量名（目的変数はそのまま）
scaled_cols = [f"{col}_scaled" for col in numeric_cols]
train_scaled = train_sorted[['time'] + scaled_cols + ['price_actual']]

<ipython-input-3-194d552869af>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_sorted[f"{col}_std_to_t"] = train_sorted[col].expanding().std(ddof=0)
<ipython-input-3-194d552869af>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_sorted[f"{col}_scaled"] = (
<ipython-input-3-194d552869af>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

In [4]:
# testの前処理
test_sorted = test.sort_values("time").reset_index(drop=True)

# trainの最終時点の平均・標準偏差を使って標準化
for col in numeric_cols:
    mu = train_sorted[f"{col}_mean_to_t"].iloc[-1]
    std = train_sorted[f"{col}_std_to_t"].iloc[-1]
    test_sorted[f"{col}_scaled"] = (test_sorted[col] - mu) / std

test_scaled = test_sorted[['time'] + [f"{col}_scaled" for col in numeric_cols]]

In [5]:
# 例：標準化後の最初の数行を表示
train_scaled.head()
test_scaled.head()

,time,generation_biomass_scaled,generation_fossil_brown_coal/lignite_scaled,generation_fossil_gas_scaled,generation_fossil_hard_coal_scaled,generation_fossil_oil_scaled,generation_hydro_pumped_storage_consumption_scaled,generation_hydro_run_of_river_and_poundage_scaled,generation_hydro_water_reservoir_scaled,generation_nuclear_scaled,...,seville_temp_max_scaled,seville_pressure_scaled,seville_humidity_scaled,seville_wind_speed_scaled,seville_wind_deg_scaled,seville_rain_1h_scaled,seville_rain_3h_scaled,seville_snow_3h_scaled,seville_clouds_all_scaled,seville_weather_id_scaled
0,2018-01-01 00:00:00+01:00,-1.336571,-1.314607,-0.718683,-1.751850,-2.124644,-0.340835,0.489439,-0.328923,0.942500,...,-1.252996,2.011949,1.044069,-0.774331,1.855445,-0.192788,-0.061253,NaN,-0.532556,0.301183
1,2018-01-01 01:00:00+01:00,-1.303199,-1.314607,-0.709440,-1.760234,-2.346720,0.950081,0.459670,-0.793849,0.938877,...,-1.474023,2.158002,1.044069,-0.774331,1.855445,-0.192788,-0.061253,NaN,-0.532556,0.301183
2,2018-01-01 02:00:00+01:00,-1.292075,-1.314607,-0.778543,-1.754316,-2.383733,2.103083,0.443432,-0.866611,0.937669,...,-1.584536,2.158002,1.044069,-0.774331,-1.434192,-0.192788,-0.061253,NaN,-0.532556,0.301183
3,2018-01-01 03:00:00+01:00,-1.325447,-1.314607,-0.831361,-1.758754,-2.383733,3.057292,0.389305,-0.996619,0.938877,...,-1.695049,2.158002,1.301429,-0.774331,-1.050561,-0.192788,-0.061253,NaN,-0.532556,0.301183
4,2018-01-01 04:00:00+01:00,-1.258703,-1.314607,-0.924232,-1.768617,-2.420746,2.734253,0.305408,-0.988059,0.938877,...,-1.584536,2.158002,1.044069,-0.774331,-1.146469,-0.192788,-0.061253,NaN,-0.532556,0.301183


In [7]:
# 🔎 処理前後の特徴量構成の変化を確認（スケーリング済データに対応）

# スケーリング前の元の数値列（price_actual除く）
numeric_cols = train.select_dtypes(include='number').columns.tolist()
if "price_actual" in numeric_cols:
    numeric_cols.remove("price_actual")

# スケーリング後の特徴量名
scaled_cols = [f"{col}_scaled" for col in numeric_cols]

# 元の特徴量数（time, price_actual除いた数値列）
original_cols = train.columns.tolist()

# スケーリング後の全列
processed_cols = train_scaled.columns.tolist()

# 新しく追加された列（スケーリング列）
new_cols = list(set(processed_cols) - set(original_cols))

# 削除された列（time以外はprice_actual程度）
removed_cols = list(set(original_cols) - set(processed_cols))

# 結果表示
print(f"🧾 元の特徴量数: {len(original_cols)}")
print(f"✅ 前処理後の特徴量数: {len(processed_cols)}")
print(f"➕ 新たに追加された列: {len(new_cols)}")
print(f"➖ 削除された列: {removed_cols}")
print(f"🧮 スケーリングされた列数: {len(scaled_cols)}")
print(f"🕒 時系列特徴量は未追加（必要であれば hour, dayofweek などを追加可）")

print("\n🎯 新規追加された特徴量例:")
print(sorted(new_cols)[:5])

🧾 元の特徴量数: 92
✅ 前処理後の特徴量数: 77
➕ 新たに追加された列: 75
➖ 削除された列: ['generation_biomass', 'seville_weather_icon', 'barcelona_rain_3h', 'madrid_weather_icon', 'valencia_weather_description', 'seville_rain_1h', 'barcelona_wind_deg', 'seville_temp_min', 'generation_other_renewable', 'bilbao_temp', 'seville_weather_description', 'generation_fossil_brown_coal/lignite', 'valencia_snow_3h', 'barcelona_weather_id', 'bilbao_weather_description', 'barcelona_weather_description', 'seville_weather_id', 'valencia_temp_min', 'bilbao_weather_id', 'seville_clouds_all', 'generation_wind_onshore', 'valencia_pressure', 'valencia_temp_max', 'total_load_actual', 'bilbao_pressure', 'madrid_temp_min', 'bilbao_wind_deg', 'madrid_clouds_all', 'generation_other', 'seville_rain_3h', 'bilbao_clouds_all', 'valencia_weather_main', 'barcelona_rain_1h', 'seville_wind_speed', 'generation_hydro_water_reservoir', 'barcelona_clouds_all', 'madrid_wind_deg', 'generation_fossil_hard_coal', 'generation_waste', 'valencia_humidity', 'madr

In [8]:
# スケーリング後の統計情報（NaNは除外）
train_scaled.describe().T[['mean', 'std', 'min', 'max']].dropna().head(10)

,mean,std,min,max
generation_biomass_scaled,-0.561276,1.029215,-7.137158,3.459581
generation_fossil_brown_coal/lignite_scaled,0.060930,0.945775,-3.121609,5.877341
generation_fossil_gas_scaled,0.265082,1.237393,-2.429652,7.611282
generation_fossil_hard_coal_scaled,-0.123404,0.948401,-3.440877,2.917286
generation_fossil_oil_scaled,-0.181724,0.896541,-5.579759,3.886432
generation_hydro_pumped_storage_consumption_scaled,-0.078092,0.968559,-2.241498,4.947803
generation_hydro_run_of_river_and_poundage_scaled,-0.202482,1.051903,-2.790590,3.075007
generation_hydro_water_reservoir_scaled,-0.168234,1.013943,-1.808765,4.017197
generation_nuclear_scaled,-0.010003,0.973666,-9.794592,4.086397
generation_other_scaled,-0.411627,1.009756,-4.459204,6.702526


## ⚙️ Step 2. 特徴量の整合性（列の追加）

In [10]:
# 特徴量列（目的変数と時間以外）を特定
scaled_feature_cols = [col for col in train_scaled.columns if col.endswith('_scaled')]

# 学習用特徴量・教師データを定義
X_train = train_scaled[scaled_feature_cols]
y_train = train_scaled['price_actual']

# テストデータにも不足列があれば追加（通常は一致しているはず）
X_test = test_scaled[scaled_feature_cols]
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0
X_test = X_test[X_train.columns]


## 📊 Step 3. モデルの学習と評価（LightGBM + 時系列CV）

In [13]:
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

In [14]:
model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    random_state=42
)

tscv = TimeSeriesSplit(n_splits=3)
rmse_scores = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(100)]
    )

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)
    print(f"Fold {fold+1} RMSE: {rmse:.4f}")

print(f"\n✅ 平均RMSE: {np.mean(rmse_scores):.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002100 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18360
[LightGBM] [Info] Number of data points in the train set: 6570, number of used features: 72
[LightGBM] [Info] Start training from score 61.611324
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 391.82
Early stopping, best iteration is:
[91]	valid_0's l2: 388.982
Fold 1 RMSE: 19.7226
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002143 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18615
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 73
[LightGBM] [Info] Start training from score 53.824721
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 212.421
Fold 2 RMSE: 14.5747

## 📤 Step 4. 予測と提出ファイルの作成

In [15]:
# テストデータ予測
y_test_pred = model.predict(X_test)

# 提出ファイル作成
submit = pd.DataFrame({
    "time": test['time'],
    "0": y_test_pred
})

In [16]:
# prompt: # 今日の日付と時刻をYYYYMMDDhhmm形式で、'submission_YYYYMMDDhhmm.csv'という名前で保存

from datetime import datetime

# 今日の日付と時刻を取得
now = datetime.now()

# ファイル名を生成
filename = f'submission_{now.strftime("%Y%m%d%H%M")}.csv'

# 提出ファイルを保存
submit.to_csv(PATH + filename, index=False, header=False)

## ✅ モデル評価コード（RMSE）

In [17]:
from sklearn.metrics import mean_squared_error
import numpy as np

# バリデーションスコア（すでに各foldのMAEは val_scores に保存済み）
# RMSE評価のために、再度 TimeSeriesSplit を使って予測・評価を行う

tscv = TimeSeriesSplit(n_splits=3)
rmse_scores = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(100)]
    )

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)
    print(f"Fold {fold+1} RMSE: {rmse:.4f}")

# 全体の平均RMSE
print(f"\n✅ 平均RMSE: {np.mean(rmse_scores):.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002054 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18360
[LightGBM] [Info] Number of data points in the train set: 6570, number of used features: 72
[LightGBM] [Info] Start training from score 61.611324
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 391.82
Early stopping, best iteration is:
[91]	valid_0's l2: 388.982
Fold 1 RMSE: 19.7226
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18615
[LightGBM] [Info] Number of data points in the train set: 13140, number of used features: 73
[LightGBM] [Info] Start training from score 53.824721
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 212.421
Fold 2 RMSE: 14.5747